In [1]:
!pip install -q efficientnet_pytorch jupytext efficientnet-pytorch catalyst pytorchcv pretrainedmodels fastprogress > /dev/null

%reset -f 
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import os
import numpy as np
import pandas as pd

import albumentations as A
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim

from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from albumentations.pytorch import ToTensorV2

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, StratifiedKFold

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings  
warnings.filterwarnings('ignore')

__print__ = print

def print(string):
    os.system(f'echo \"{string}\"')
    __print__(string)
    
from fastprogress.fastprogress import master_bar, progress_bar    

In [2]:
DIR_INPUT = './plant-pathology-2020-fgvc7'

SEED = 42
N_FOLDS = 5
N_EPOCHS = 15
BATCH_SIZE = 12
SIZE = 512

In [3]:
class PlantDataset(Dataset):
    
    def __init__(self, df, transforms=None):
    
        self.df = df
        self.transforms=transforms
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        image_src = DIR_INPUT + '/images/' + self.df.loc[idx, 'image_id'] + '.jpg'
        # print(image_src)
        image = cv2.imread(image_src, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        labels = self.df.loc[idx, ['healthy', 'multiple_diseases', 'rust', 'scab']].values
        labels = torch.from_numpy(labels.astype(np.int8))
        labels = labels.unsqueeze(-1)
        
        if self.transforms:
            transformed = self.transforms(image=image)
            image = transformed['image']

        return image, labels

In [4]:
from torch import nn, optim
from glob import glob 

dropout = torch.nn.Dropout(p=0.30)
relu=torch.nn.LeakyReLU()
pool = nn.MaxPool2d(2, 2)

class ConvRes(nn.Module):
    def __init__(self, insize, outsize):
        super(ConvRes, self).__init__()
        drate = .3
        self.math = nn.Sequential(
            nn.BatchNorm2d(insize),
            nn.Dropout(drate),
            torch.nn.Conv2d(insize, outsize, kernel_size=2, padding=2),
            nn.PReLU(),
        )

    def forward(self, x):
        return self.math(x)


class ConvCNN(nn.Module):
    def __init__(self, insize, outsize, kernel_size=7, padding=2, pool=2, avg=True):
        super(ConvCNN, self).__init__()
        self.avg = avg
        self.math = torch.nn.Sequential(
            torch.nn.Conv2d(insize, outsize, kernel_size=kernel_size, padding=padding),
            torch.nn.BatchNorm2d(outsize),
            torch.nn.LeakyReLU(),
            torch.nn.MaxPool2d(pool, pool),
        )
        self.avgpool = torch.nn.AvgPool2d(pool, pool)

    def forward(self, x):
        x = self.math(x)
        if self.avg is True:
            x = self.avgpool(x)
        return x


    
class SimpleNet(nn.Module):
    def __init__(self,num_classes, n_dim):
        super(SimpleNet, self).__init__()
        self.num_classes=num_classes
        self.avgpool = nn.AdaptiveAvgPool2d(1)

        self.cnn1 = ConvCNN (n_dim,32,  kernel_size=7, pool=4, avg=False)
        self.cnn2 = ConvCNN (32,32, kernel_size=5, pool=2, avg=True)
        self.cnn3 = ConvCNN (32,32, kernel_size=5, pool=2, avg=True)

        self.res1 = ConvRes (32,64)

        self.features = nn.Sequential(
            self.cnn1, dropout,
            self.cnn2,
            self.cnn3,
            self.res1,
        )

        self.classifier = torch.nn.Sequential(
            nn.Linear(6400, (num_classes)), # 2305 for 256
        )

        self.sig=nn.Sigmoid()

# torch.Size([16, 3, 256, 256])
# torch.Size([16, 2304])

    def forward(self, x):
#         print (x.data.shape)
        x = self.features(x)
        x = x.view(x.size(0), -1)
#         print (x.data.shape)
        x = self.classifier(x)
        if (self.num_classes == 1):
                x = self.sig(x)
        return x

    #         return F.log_softmax(x)


def simpleXX_generic(n_classes: int = 4,imgDim:int=3):
    # depth, num_classes = 1, widen_factor = 1, dropRate = 0.0
    model = SimpleNet(num_classes=n_classes, n_dim=imgDim)  # 56    
    return model

class PlantModel(nn.Module):
    
    def __init__(self, num_classes=4):
        super().__init__()
        
        self.backbone = torchvision.models.resnet18(pretrained=True)
        
        in_features = self.backbone.fc.in_features

        self.logit = nn.Linear(in_features, num_classes)
        
    def forward(self, x):
        batch_size, C, H, W = x.shape
        
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)
        
        x = F.adaptive_avg_pool2d(x,1).reshape(batch_size,-1)
        x = F.dropout(x, 0.25, self.training)

        x = self.logit(x)

        return x


In [5]:
transforms_train = A.Compose([
    A.RandomResizedCrop(height=SIZE, width=SIZE, p=1.0),
    A.Flip(),
    A.ShiftScaleRotate(rotate_limit=1.0, p=0.8),

    # Pixels
    A.OneOf([
        A.IAAEmboss(p=1.0),
        A.IAASharpen(p=1.0),
        A.Blur(p=1.0),
    ], p=0.5),

    # Affine
    A.OneOf([
        A.ElasticTransform(p=1.0),
        A.IAAPiecewiseAffine(p=1.0)
    ], p=0.5),

    A.Normalize(p=1.0),
    ToTensorV2(p=1.0),
])

transforms_valid = A.Compose([
    A.Resize(height=SIZE, width=SIZE, p=1.0),
    A.Normalize(p=1.0),
    ToTensorV2(p=1.0),
])

In [6]:
submission_df = pd.read_csv(DIR_INPUT + '/sample_submission.csv')
submission_df.iloc[:, 1:] = 0

submission_df.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,0,0,0,0
1,Test_1,0,0,0,0
2,Test_2,0,0,0,0
3,Test_3,0,0,0,0
4,Test_4,0,0,0,0


In [7]:
dataset_test = PlantDataset(df=submission_df, transforms=transforms_valid)
dataloader_test = DataLoader(dataset_test, batch_size=BATCH_SIZE, num_workers=4, shuffle=False)

In [8]:
train_df = pd.read_csv(DIR_INPUT + '/train.csv')

# For debugging.
# train_df = train_df.sample(n=100)
# train_df.reset_index(drop=True, inplace=True)

train_labels = train_df.iloc[:, 1:].values

# Need for the StratifiedKFold split
train_y = train_labels[:, 2] + train_labels[:, 3] * 2 + train_labels[:, 1] * 3

train_df.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0


In [9]:
folds = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
oof_preds = np.zeros((train_df.shape[0], 4))

In [10]:
# Download pretrained weights.
# model = 
# model=
# model 
models=[PlantModel()]

In [11]:
class DenseCrossEntropy(nn.Module):

    def __init__(self):
        super(DenseCrossEntropy, self).__init__()
        
        
    def forward(self, logits, labels):
        logits = logits.float()
        labels = labels.float()
        
        logprobs = F.log_softmax(logits, dim=-1)
        
        loss = -labels * logprobs
        loss = loss.sum(-1)

        return loss.mean()
    

In [12]:
from tqdm import tqdm 
def train_one_fold(i_fold, model, criterion, optimizer, dataloader_train, dataloader_valid):
    
    train_fold_results = []
    
     #Creating progress bar
    mb = master_bar(range(N_EPOCHS))
    mb.write(['epoch','train_loss','valid_loss','val_roc'],table=True)
    for epoch in mb:   
#     for epoch in  tqdm (range(N_EPOCHS)):
        # print('  Epoch {}/{}'.format(epoch + 1, N_EPOCHS))
        # print('  ' + ('-' * 20))
#         os.system(f'echo \"  Epoch {epoch}\"')

        model.train()
        tr_loss = 0

        for step, batch in enumerate(dataloader_train):

            images = batch[0]
            labels = batch[1]

            images = images.to(device, dtype=torch.float)
            labels = labels.to(device, dtype=torch.float)
            
            outputs = model(images)
            loss = criterion(outputs, labels.squeeze(-1))                
            loss.backward()

            tr_loss += loss.item()

            optimizer.step()
            optimizer.zero_grad()

        # Validate
        model.eval()
        val_loss = 0
        val_preds = None
        val_labels = None

        for step, batch in enumerate(dataloader_valid):

            images = batch[0]
            labels = batch[1]

            if val_labels is None:
                val_labels = labels.clone().squeeze(-1)
            else:
                val_labels = torch.cat((val_labels, labels.squeeze(-1)), dim=0)

            images = images.to(device, dtype=torch.float)
            labels = labels.to(device, dtype=torch.float)

            with torch.no_grad():
                outputs = model(images)

                loss = criterion(outputs, labels.squeeze(-1))
                val_loss += loss.item()

                preds = torch.softmax(outputs, dim=1).data.cpu()

                if val_preds is None:
                    val_preds = preds
                else:
                    val_preds = torch.cat((val_preds, preds), dim=0)
        
        trn_loss=tr_loss / len(dataloader_train)
        val_loss=val_loss / len(dataloader_valid)
        val_roc=roc_auc_score(val_labels, val_preds, average='macro')
        
        mb.write([epoch,f'{trn_loss:.6f}',f'{val_loss:.6f}',f'{val_roc:.6f}'],table=True)
        train_fold_results.append({
            'fold': i_fold,
            'epoch': epoch,
            'train_loss': trn_loss,
            'valid_loss': val_loss,
            'valid_score':val_roc ,
        })

    return val_preds, train_fold_results

In [13]:
from tqdm import tqdm 
submissions = None
train_results = []

for m in models:
    for i_fold, (train_idx, valid_idx) in tqdm (enumerate(folds.split(train_df, train_y))):
        print("Fold {}/{}".format(i_fold + 1, N_FOLDS))

        valid = train_df.iloc[valid_idx]
        valid.reset_index(drop=True, inplace=True)

        train = train_df.iloc[train_idx]
        train.reset_index(drop=True, inplace=True)    

        dataset_train = PlantDataset(df=train, transforms=transforms_train) # shlomo
        dataset_valid = PlantDataset(df=valid, transforms=transforms_valid)

        dataloader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, num_workers=8, shuffle=True)
        dataloader_valid = DataLoader(dataset_valid, batch_size=BATCH_SIZE, num_workers=8, shuffle=False)

        device = torch.device("cuda:0")

        model = m
        model.to(device)

        criterion = DenseCrossEntropy()
        plist = [{'params': model.parameters(), 'lr': 5e-5}]
        optimizer = optim.Adam(plist, lr=5e-5)

        val_preds, train_fold_results = train_one_fold(i_fold, model, criterion, optimizer, dataloader_train, dataloader_valid)
        oof_preds[valid_idx, :] = val_preds.numpy()

        train_results = train_results + train_fold_results

        model.eval()
        test_preds = None

        for step, batch in (enumerate(dataloader_test)):

            images = batch[0]
            images = images.to(device, dtype=torch.float)

            with torch.no_grad():
                outputs = model(images)

                if test_preds is None:
                    test_preds = outputs.data.cpu()
                else:
                    test_preds = torch.cat((test_preds, outputs.data.cpu()), dim=0)


        # Save predictions per fold
        submission_df[['healthy', 'multiple_diseases', 'rust', 'scab']] = torch.softmax(test_preds, dim=1)
        submission_df.to_csv('submission_fold_{}.csv'.format(i_fold), index=False)

        # logits avg
        if submissions is None:
            submissions = test_preds / N_FOLDS
        else:
            submissions += test_preds / N_FOLDS

    print("5-Folds CV score: {:.4f}".format(roc_auc_score(train_labels, oof_preds, average='macro')))

0it [00:00, ?it/s]

Fold 1/5


epoch,train_loss,valid_loss,val_roc
0,0.703458,0.305305,0.955184
1,0.454628,0.270652,0.949448
2,0.416272,0.232505,0.957797
3,0.385335,0.189904,0.977366
4,0.331392,0.231216,0.958305
5,0.342648,0.203714,0.970655
6,0.337041,0.211160,0.972009
7,0.351580,0.198341,0.972083
8,0.274910,0.187475,0.970406
9,0.265756,0.167074,0.979077


1it [20:13, 1213.29s/it]

Fold 2/5


...

4it [1:21:11, 1217.40s/it]

Fold 5/5


epoch,train_loss,valid_loss,val_roc
0,0.165056,0.039629,0.999716
1,0.148191,0.026568,0.999878
2,0.136648,0.018200,0.999983
3,0.119099,0.033892,0.999681
4,0.135100,0.035183,0.999847
5,0.145066,0.025359,0.999694
6,0.137850,0.027839,0.999498
7,0.141281,0.025306,0.999923
8,0.119714,0.028048,0.999876
9,0.151964,0.042910,0.999090


5it [1:41:33, 1218.76s/it]

5-Folds CV score: 0.9910


# Show train history

In [14]:
train_results = pd.DataFrame(train_results)
train_results.head(10)

,fold,epoch,train_loss,valid_loss,valid_score
0,0,0,0.703458,0.305305,0.955184
1,0,1,0.454628,0.270652,0.949448
2,0,2,0.416272,0.232505,0.957797
3,0,3,0.385335,0.189904,0.977366
4,0,4,0.331392,0.231216,0.958305
5,0,5,0.342648,0.203714,0.970655
6,0,6,0.337041,0.211160,0.972009
7,0,7,0.351580,0.198341,0.972083
8,0,8,0.274910,0.187475,0.970406
9,0,9,0.265756,0.167074,0.979077


In [19]:
fig = make_subplots(rows=2, cols=1)

colors = [
    ('#d32f2f', '#ef5350'),
    ('#303f9f', '#5c6bc0'),
    ('#00796b', '#26a69a'),
    ('#fbc02d', '#ffeb3b'),
    ('#5d4037', '#8d6e63'),
]

for k in models:
    
    for i in range(N_FOLDS):
        data = train_results[train_results['fold'] == i]

        fig.add_trace(go.Scatter(x=data['epoch'].values,
                                 y=data['train_loss'].values,
                                 mode='lines',
                                 visible='legendonly' if i > 0 else True,
                                 line=dict(color=colors[i][0], width=2),
                                 name='Train loss - Fold #{}'.format(i)),
                     row=1, col=1)

        fig.add_trace(go.Scatter(x=data['epoch'],
                                 y=data['valid_loss'].values,
                                 mode='lines+markers',
                                 visible='legendonly' if i > 0 else True,
                                 line=dict(color=colors[i][1], width=2),
                                 name='Valid loss - Fold #{}'.format(i)),
                     row=1, col=1)

        fig.add_trace(go.Scatter(x=data['epoch'].values,
                                 y=data['valid_score'].values,
                                 mode='lines+markers',
                                 line=dict(color=colors[i][0], width=2),
                                 name='Valid score - Fold #{}'.format(i),
                                 showlegend=False),
                     row=2, col=1)

fig.update_layout({
  "annotations": [
    {
      "x": 0.225, 
      "y": 1.0, 
      "font": {"size": 16}, 
      "text": "Train / valid losses", 
      "xref": "paper", 
      "yref": "paper", 
      "xanchor": "center", 
      "yanchor": "bottom", 
      "showarrow": False
    }, 
    {
      "x": 0.775, 
      "y": 1.0, 
      "font": {"size": 16}, 
      "text": "Validation scores", 
      "xref": "paper", 
      "yref": "paper", 
      "xanchor": "center", 
      "yanchor": "bottom", 
      "showarrow": False
    }, 
  ]
})

fig.show()

# Generate submission

In [16]:
submission_df[['healthy', 'multiple_diseases', 'rust', 'scab']] = torch.softmax(submissions, dim=1)
submission_df.to_csv('submission.csv', index=False)

In [17]:
submission_df

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,2.786740e-06,0.338050,0.661875,0.000072
1,Test_1,3.807185e-05,0.044647,0.953531,0.001783
2,Test_2,3.421972e-06,0.000059,0.000010,0.999927
3,Test_3,9.920746e-01,0.000012,0.007582,0.000331
4,Test_4,1.356204e-09,0.001099,0.998897,0.000004
...,...,...,...,...,...
1816,Test_1816,1.474522e-07,0.001901,0.998086,0.000013
1817,Test_1817,2.455675e-01,0.265647,0.009253,0.479532
1818,Test_1818,4.602588e-04,0.068725,0.930749,0.000066
1819,Test_1819,9.994057e-01,0.000068,0.000057,0.000469
